In [ ]:
import sys
import os
sys.path.append(os.path.abspath('.'))

In [ ]:
import lib as grin

In [ ]:
from lib import datasets

In [ ]:
from lib.utils import numpy_metrics
from lib.utils.parser_utils import str_to_bool

metrics = {
    'mae': numpy_metrics.masked_mae,
    'mse': numpy_metrics.masked_mse,
    'mre': numpy_metrics.masked_mre,
    'mape': numpy_metrics.masked_mape
}


In [ ]:
idxs[1]

In [ ]:
122 / 24

In [ ]:
import pandas as pd
df_air = pd.HDFStore('/home/jhzhou/repos/tourism-imputation/data/grin-data/air_quality/small36.h5')

In [ ]:
df_air.groups()

In [ ]:
df_air['eval_mask']

In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from pathlib import Path

data_path = Path('../../data/open-data/HK2012-2018/')
df_au = pd.read_csv(data_path / 'Australia.csv')
df_au['date'] = pd.to_datetime(df_au['date'])
df_au.set_index('date', inplace=True)
df_au.head()

## Dataset Test

In [ ]:
import numpy as np
import pandas as pd
import lib as grin
from lib.datasets.pd_dataset import PandasDataset
from lib.utils import compute_mean
dataset_path = '../../data/open-data/HK2012-2018/Australia.csv'
mask_path = '../../data/masks/block5.npy'

country = 'au'

class ArrivalDataset(PandasDataset):
    SEED = 1145
    
    def __init__(self, impute_nans=True, small=False, freq='MS'):
        self.random = np.random.default_rng(self.SEED)
        self.eval_mask = None
        df, mask = self.load(impute_nans=impute_nans, small=small)
        df = df.astype('float32')
        super().__init__(dataframe=df, u=None, mask=mask, name='arrival', freq=freq, aggr='nearest')

    def load_raw(self, small=False):
        df = pd.read_csv(dataset_path, index_col=0, parse_dates=True)
        mask = np.load(mask_path)
        return df, mask


    def load(self, impute_nans=True, small=False):
        # load readings and stations metadata
        df, eval_mask = self.load_raw(small)
        # compute the masks
        mask = (~np.isnan(df.values)).astype('uint8')  # 1 if value is not nan else 0

        eval_mask = eval_mask.astype('uint8')
        self.eval_mask = eval_mask  # 1 if value is ground-truth for imputation else 0
        # eventually replace nans with weekly mean by hour
        if impute_nans:
            df = df.fillna(compute_mean(df))
        # compute distances from latitude and longitude degrees
        return df, mask

    def splitter(self, dataset, val_len=1., in_sample=False, window=0):
        if in_sample:
            train_idxs = np.arange(len(dataset))
        else:
            val_len = 12
            test_len = 24
            train_idxs = np.arange(len(dataset) - val_len - test_len)
            val_idxs = np.arange(len(train_idxs), len(dataset) - test_len)
            test_idxs = np.arange(len(dataset) - test_len, len(dataset))
        return [train_idxs, val_idxs, test_idxs]

    def get_similarity(self):
        N = len(self.df)
        return np.ones(N, N) - np.identity(N) 

    @property
    def mask(self):
        return self._mask

    @property
    def training_mask(self):
        return self._mask if self.eval_mask is None else (self._mask & (1 - self.eval_mask))

    def test_interval_mask(self, dtype=bool, squeeze=True):
        m = np.in1d(self.df.index.month, self.test_months).astype(dtype)
        if squeeze:
            return m
        return m[:, None]

In [ ]:
arrival = ArrivalDataset()

In [ ]:
from lib.datasets import AirQuality
air = AirQuality()

In [ ]:
arrival.numpy().dtype

## Splitter Test

In [ ]:
dataset = arrival

In [ ]:
from lib.data.imputation_dataset import ImputationDataset, GraphImputationDataset
from lib.nn import models
from lib import fillers, datasets, config

def has_graph_support(model_cls):
    return model_cls in [models.GRINet, models.MPGRUNet, models.BiMPGRUNet]
model_cls, filler_cls = models.GRINet, fillers.GraphFiller
dataset_cls = GraphImputationDataset if has_graph_support(model_cls) else ImputationDataset
torch_dataset = dataset_cls(*dataset.numpy(return_idx=True),
                            mask=dataset.training_mask,
                            eval_mask=dataset.eval_mask,
                            window=12,
                            stride=1)

idxs = dataset.splitter(torch_dataset, val_len=0.1, in_sample=False, window=0)

In [ ]:
torch_dataset[0][0]['x'].shape

In [ ]:
idxs

In [ ]:
torch_dataset.data

# Mask Test

In [ ]:
import lib as grin
from lib.utils import sample_mask
import pandas as pd

arrival = pd.read_csv('/home/jhzhou/repos/tourism-imputation/data/open-data/HK2012-2018/Australia.csv', index_col=0, parse_dates=True)
shape = arrival.shape

In [ ]:
random5 = sample_mask(shape,p = 0.05, min_seq=1, max_seq=1)
random10 = sample_mask(shape,p = 0.10, min_seq=1, max_seq=1)
block5 = sample_mask(shape,p = 0.006, min_seq =6, max_seq=12)
block10 = sample_mask(shape,p = 0.012, min_seq=6, max_seq=12)

In [ ]:
import numpy as np
np.save('../../data/masks/random5.npy',random5)
np.save('../../data/masks/random10.npy',random10)
np.save('../../data/masks/block5.npy',block5)
np.save('../../data/masks/block10.npy',block10)